In [2]:
import glob
import pandas as pd
import time
import datetime as dt
import os
import numpy as np
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pathlib
from pathlib import Path
import pickle
import glob
from datetime import datetime
warnings.filterwarnings("ignore")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
import math
import plotly.express as px
import sys
from scipy.stats import skew, kurtosis
from scipy.stats import spearmanr
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.metrics import mean_tweedie_deviance
import sys
import umap as umap
def mean_absolute_percentage_error(y_true, y_predicted):
    y_true, y_predicted = np.array(y_true), np.array(y_predicted)
    return np.mean(np.abs((y_true - y_predicted) / y_true)) * 100
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
os.chdir('/content')

FileNotFoundError: [WinError 2] The system cannot find the file specified: '/content'

In [ ]:
import pandas as pd

import numpy as np

data = np.genfromtxt('/content/Precip_Raw_Data.csv', delimiter=',', dtype=str, encoding='utf-8')
Agg_data = pd.DataFrame(data[1:], columns=data[0])  # Adjust header if necessary


In [ ]:
Agg_data['Timestamp_To'].max()

'2024-01-19 22:00:00'

In [ ]:
Agg_data.tail()

,Timestamp_From,Timestamp_To,Special_Filling_temp,Special_Filling_Flow,First_Agglo_Tank_Temp_T42,Tank_47_Temp,PPT-48_Temperature,Fine_Seed_Tonnage,Fine_Seed_Flow,Fine_Seed_Density,Fine_Seed_Charge,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp,PHE_Outlet_Temp,Regular_Filling_Flow,Tank__52_Temp,Tank__56_Temp,Coarse_Seed_1_Tonnage,Coarse_Seed_2_Tonnage,Coarse_Seed_2_Flow,Coarse_Seed_1_Density,Coarse_Seed_2_Density,HAT_3_5u,HAT_45u,HAT_D50,Filling_Concentration,Filling_Ratio,Batch_Circulation_Hrs,Feed_Hydrate_D50,Feed_Hydrate_SSA,Date
9159,2024-01-19 17:00:00,2024-01-19 18:00:00,97.01,137.22,82.49,77.44,54.67,38.34,100.0,1.44,0.0,,,,,74.79,75.57,166.46,66.17,65.42,152.34,178.66,209.15,1.6,1.66,,,,231.8,0.659,,,,2024-01-19
9160,2024-01-19 18:00:00,2024-01-19 19:00:00,96.0,141.38,82.39,77.46,54.57,36.95,100.01,1.43,0.0,,,,,74.39,75.18,163.94,65.78,65.35,152.06,138.89,210.51,1.6,1.57,,,,,,,,,2024-01-19
9161,2024-01-19 19:00:00,2024-01-19 20:00:00,96.38,149.86,82.53,77.56,54.41,36.84,99.99,1.43,0.0,,,,,74.55,75.34,180.81,66.43,65.33,173.66,157.34,209.51,1.65,1.61,,,,,,,,,2024-01-19
9162,2024-01-19 20:00:00,2024-01-19 21:00:00,96.02,140.27,82.31,77.53,54.03,35.37,97.39,1.42,0.0,,,,,75.35,76.16,190.03,66.39,65.35,142.22,165.68,209.83,1.58,1.63,,,,,,,,,2024-01-19
9163,2024-01-19 21:00:00,2024-01-19 22:00:00,95.01,174.21,82.15,77.68,53.83,41.89,110.36,1.43,0.0,0.337,1.8,76.3,30.2,75.91,76.73,182.66,66.75,65.37,159.9,138.34,210.74,1.62,1.57,0.9,46.3,48.6,231.1,0.67,,109.0,0.07,2024-01-19


In [ ]:
Agg_data.drop(['Timestamp_From','Date','Fine_Seed_Charge','Batch_Circulation_Hrs','Tank_47_Temp','PPT-48_Temperature'], axis = 1,inplace = True)

In [ ]:
final_Agg_data=Agg_data[['Timestamp_To','Special_Filling_Flow','Fine_Seed_Tonnage','Fine_Seed_Flow','Fine_Seed_Density',
                        'Fine_Seed_SSA','Fine_Seed_3_5u','Fine_Seed_45u','Fine_Seed_D50','PHE_Inlet_Temp','Regular_Filling_Flow',
                        'Coarse_Seed_1_Tonnage','Coarse_Seed_2_Flow','HAT_3_5u','HAT_45u','HAT_D50','Filling_Ratio','Feed_Hydrate_D50']]

In [ ]:
def replace_outliers_with_median(data):
    # Convert relevant columns to numeric before calculating quantiles
    numeric_cols = data.select_dtypes(include=np.number).columns
    # Convert only numeric_cols to numeric, excluding Timestamp_To
    data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Ensure 'Timestamp_To' is excluded from quantile calculations
    numeric_data = data.select_dtypes(include=np.number)

    Q1 = numeric_data.quantile(0.25)  # Apply quantile to numeric_data
    Q3 = numeric_data.quantile(0.75)  # Apply quantile to numeric_data
    IQR = Q3 - Q1
    median = numeric_data.median()  # Calculate median for numeric_data
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Apply outlier replacement only to numeric columns
    for col in numeric_cols:
        data[col] = data[col].where((data[col].between(lower_bound[col], upper_bound[col])) | data[col].isna(), median[col])

    return data

In [ ]:
treated_agg_data = replace_outliers_with_median(final_Agg_data)

In [ ]:
min_timestamp = treated_agg_data['Timestamp_To'].min()
print(min_timestamp)
# Convert min_timestamp to datetime object
min_timestamp = pd.to_datetime(min_timestamp)
timestamp_1hr_before = min_timestamp - pd.Timedelta(hours=1)
timestamp_2hr_before = min_timestamp - pd.Timedelta(hours=2)

2023-01-01 01:00:00


In [ ]:
new_rows_dynamic = [
    {col: timestamp_1hr_before if col == 'Timestamp_To' else pd.NA for col in treated_agg_data.columns},
    {col: timestamp_2hr_before if col == 'Timestamp_To' else pd.NA for col in treated_agg_data.columns}
]

new_rows_df_dynamic = pd.DataFrame(new_rows_dynamic)

In [ ]:
data_extended_dynamic = pd.concat([new_rows_df_dynamic, treated_agg_data], ignore_index=True)
# Convert 'Timestamp_To' to datetime objects before sorting
data_extended_dynamic['Timestamp_To'] = pd.to_datetime(data_extended_dynamic['Timestamp_To'])
data_extended_dynamic.sort_values(by='Timestamp_To', inplace=True)
data_extended_dynamic.reset_index(drop=True, inplace=True)

In [ ]:
def initial_data_preprocessing(treated_agg_data):
    # Convert 'Timestamp_To' to datetime objects before finding the minimum
    treated_agg_data['Timestamp_To'] = pd.to_datetime(treated_agg_data['Timestamp_To'])
    min_timestamp = treated_agg_data['Timestamp_To'].min()
    print(min_timestamp)
    timestamp_1hr_before = min_timestamp - pd.Timedelta(hours=1)
    timestamp_2hr_before = min_timestamp - pd.Timedelta(hours=2)
    new_rows_dynamic = [
    {col: timestamp_1hr_before if col == 'Timestamp_To' else pd.NA for col in treated_agg_data.columns},
    {col: timestamp_2hr_before if col == 'Timestamp_To' else pd.NA for col in treated_agg_data.columns}]
    new_rows_df_dynamic = pd.DataFrame(new_rows_dynamic)
    data_extended_dynamic = pd.concat([new_rows_df_dynamic, treated_agg_data], ignore_index=True)
    data_extended_dynamic.sort_values(by='Timestamp_To', inplace=True)
    data_extended_dynamic.reset_index(drop=True, inplace=True)
    data_extended_dynamic['Timestamp_To'] = data_extended_dynamic['Timestamp_To'] + pd.Timedelta(hours=1)
    data_extended_dynamic.set_index('Timestamp_To', inplace=True)

    return data_extended_dynamic

In [ ]:
df=initial_data_preprocessing(treated_agg_data)

2023-01-01 01:00:00


In [ ]:
columns_to_aggregate = ['Special_Filling_temp','Special_Filling_Flow','First_Agglo_Tank_Temp_T42',
                        'Fine_Seed_Tonnage','Fine_Seed_Flow','Fine_Seed_Density',
                       'PHE_Inlet_Temp','PHE_Outlet_Temp','Regular_Filling_Flow','Tank__52_Temp','Tank__56_Temp',
                        'Coarse_Seed_1_Tonnage','Coarse_Seed_2_Tonnage','Coarse_Seed_2_Flow','Coarse_Seed_1_Density',
                        'Coarse_Seed_2_Density',]
columns_to_resample = ['Fine_Seed_SSA','Fine_Seed_3_5u','Fine_Seed_45u','Fine_Seed_D50','HAT_3_5u','HAT_45u','HAT_D50',
                       'Filling_Concentration','Filling_Ratio','Feed_Hydrate_D50','Feed_Hydrate_SSA']

In [ ]:
def data_aggregation(df):
    resampled_df = pd.DataFrame()
    for feature in df.columns:
        if feature in columns_to_aggregate:
            for agg_name, agg_func in agg_functions.items():
                new_feature_name = f'{feature}_{agg_name}'
                # Convert the column to numeric before applying aggregation
                # errors='coerce' converts invalid values to NaN
                resampled_df[new_feature_name] = pd.to_numeric(df[feature], errors='coerce').resample(resample_frequency).apply(agg_func)
        elif feature in columns_to_resample:
            # Convert the column to numeric before applying aggregation
            # errors='coerce' converts invalid values to NaN
            resampled_df[feature] = pd.to_numeric(df[feature], errors='coerce').resample(resample_frequency).mean()
    resampled_df.reset_index(inplace=True)
    resampled_df['Timestamp_To'] = resampled_df['Timestamp_To'] + pd.Timedelta(hours=6)
    return resampled_df

In [ ]:
resample_frequency = '8H'
agg_functions = {
    'min': 'min',
    'max': 'max',
    'median': 'median',
    'mean': 'mean',
    'std': 'std',
    'var': 'var',
    'var_coeff': lambda x: x.std() / x.mean() if x.mean() != 0 else 0
}

In [ ]:
resampled_df=data_aggregation(df)

In [ ]:
resampled_df.tail()

,Timestamp_To,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50
1147,2024-01-18 14:00:00,137.86,234.90,204.05,195.29500,36.413710,1325.958286,0.186455,34.06,47.14,40.180,40.69375,3.769031,14.205598,0.092619,78.60,101.23,90.02,89.08250,6.572059,43.191964,0.073775,1.46,1.49,1.465,1.46875,0.011260,0.000127,0.007666,0.388,2.0,85.7,25.185,71.62,82.82,75.130,76.04875,3.273759,10.717498,0.043048,2.85,106.26,81.225,75.40125,31.692815,1004.434498,0.420322,124.23,176.00,143.670,145.78625,18.732282,350.898398,0.128491,198.13,205.12,200.250,200.78375,2.096807,4.396598,0.010443,0.6,23.6,84.8,0.663333,116.6
1148,2024-01-18 22:00:00,126.50,146.69,131.74,132.67250,6.316275,39.895336,0.047608,41.00,45.88,44.475,44.04875,1.822608,3.321898,0.041377,89.11,90.01,89.98,89.87250,0.308996,0.095479,0.003438,1.47,1.50,1.490,1.48625,0.011877,0.000141,0.007991,0.318,1.7,69.9,33.200,77.70,91.59,85.055,84.61875,5.580324,31.140012,0.065947,40.41,228.00,144.780,145.76125,55.988756,3134.740784,0.384113,79.60,173.23,138.940,132.48875,31.954183,1021.069813,0.241184,163.94,247.05,194.770,198.60625,27.526477,757.706912,0.138598,0.6,24.7,84.2,0.662000,111.2
1149,2024-01-19 06:00:00,123.03,152.11,131.57,133.95875,8.946946,80.047841,0.066789,39.10,44.61,41.095,41.05500,1.947049,3.791000,0.047425,89.97,99.22,90.05,91.65375,3.212253,10.318570,0.035048,1.45,1.49,1.465,1.46500,0.013093,0.000171,0.008937,0.310,1.6,69.7,33.600,75.14,78.19,76.155,76.41000,1.090269,1.188686,0.014269,127.27,256.68,201.000,196.34000,47.883435,2292.823371,0.243880,110.62,170.13,152.455,147.52625,18.668833,348.525312,0.126546,189.50,229.30,210.990,210.95250,13.758221,189.288650,0.065220,0.4,19.2,88.0,0.657500,117.4
1150,2024-01-19 14:00:00,125.50,144.22,134.76,133.99500,6.195337,38.382200,0.046236,38.12,41.28,38.670,39.18875,1.140394,1.300498,0.029100,89.97,100.04,90.23,92.20375,3.558314,12.661598,0.038592,1.45,1.46,1.460,1.45750,0.004629,0.000021,0.003176,0.334,1.8,73.2,31.165,75.10,76.22,75.495,75.55125,0.368411,0.135727,0.004876,131.55,181.21,169.525,160.18750,22.857566,522.468336,0.142693,110.82,202.12,160.385,159.41500,30.581229,935.211543,0.191834,207.21,237.00,215.075,220.07875,12.547881,157.449327,0.057015,0.0,15.9,89.9,0.660667,115.0
1151,2024-01-19 22:00:00,137.22,174.21,141.09,145.73375,12.130863,147.157827,0.083240,35.37,41.89,38.005,38.03625,1.907518,3.638627,0.050150,97.39,110.36,100.00,100.96875,3.903008,15.233470,0.038656,1.42,1.44,1.430,1.43125,0.006409,0.000041,0.004478,0.337,1.8,76.3,30.200,74.39,75.91,75.100,75.12375,0.556390,0.309570,0.007406,86.81,190.03,165.200,155.24875,34.369680,1181.274

In [ ]:
def data_filter(resampled_df):
    resampled_df_sorted = resampled_df.sort_values(by='Timestamp_To', ascending=False)
    for index, row in resampled_df_sorted.head(5).iterrows():
        if pd.isna(row['Feed_Hydrate_D50']):
            resampled_df_sorted = resampled_df_sorted.drop(index)
    resampled_df = resampled_df_sorted.sort_values(by='Timestamp_To')
    return resampled_df

In [ ]:
filtered_df=data_filter(resampled_df)

In [ ]:
filtered_df=filtered_df[filtered_df['Timestamp_To']<="2024-01-04 22:00:00"]

In [ ]:
filtered_df.tail()

,Timestamp_To,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50
1102,2024-01-03 14:00:00,164.30,232.93,183.920,195.84625,28.057475,787.221912,0.143263,23.05,43.94,27.395,29.96125,7.296235,53.235041,0.243522,89.89,90.11,90.050,90.03250,0.078331,0.006136,0.000870,1.37,1.49,1.400,1.41125,0.041209,0.001698,0.029201,0.176,0.9,31.8,67.900000,60.68,73.61,71.880,70.30375,4.429256,19.618313,0.063002,37.12,231.20,95.115,111.00250,62.902935,3956.779250,0.566680,138.25,224.06,195.365,187.57875,30.251320,915.142355,0.161273,157.65,252.64,217.270,213.89125,29.853118,891.208641,0.139571,0.0,0.00,103.0,0.67000,105.0
1103,2024-01-03 22:00:00,150.07,187.79,164.255,167.34875,12.825173,164.485070,0.076637,28.59,36.39,33.845,33.69750,2.498375,6.241879,0.074141,89.00,90.95,90.010,89.99625,0.523776,0.274341,0.005820,1.40,1.45,1.435,1.43250,0.014880,0.000221,0.010388,0.154,0.8,19.9,72.800000,60.94,74.01,70.870,69.03375,5.311010,28.206827,0.076934,60.83,197.63,122.645,127.81625,47.155962,2223.684712,0.368936,157.80,193.42,171.570,173.20500,14.721896,216.734229,0.084997,176.92,254.43,208.335,208.54875,23.309149,543.316413,0.111768,0.0,1.00,93.0,0.66900,112.0
1104,2024-01-04 06:00:00,160.16,206.59,176.500,178.42625,14.855285,220.679484,0.083257,25.05,32.33,30.445,29.49375,2.592489,6.720998,0.087900,88.85,90.08,89.965,89.73875,0.498525,0.248527,0.005555,1.38,1.43,1.410,1.40750,0.016690,0.000279,0.011858,0.168,0.8,28.0,59.000000,63.02,74.88,69.245,69.04500,4.966629,24.667400,0.071933,69.89,133.15,115.035,109.91500,20.714797,429.102829,0.188462,144.05,185.45,157.795,160.10375,13.762329,189.401712,0.085959,107.90,263.98,213.010,202.44875,51.791141,2682.322327,0.255823,0.0,0.00,102.0,0.67450,105.0
1105,2024-01-04 14:00:00,155.95,206.67,172.235,176.65750,17.148238,294.062050,0.097071,22.25,24.86,24.040,23.69375,1.113244,1.239312,0.046985,89.93,110.28,90.060,95.34000,7.874701,62.010914,0.082596,1.36,1.38,1.370,1.37125,0.006409,0.000041,0.004674,0.215,1.1,42.5,54.786667,60.34,77.26,73.855,71.18250,6.664826,44.419907,0.093630,72.26,184.38,107.385,116.31125,42.222794,1782.764298,0.363016,125.88,181.55,147.805,150.48375,21.917718,480.386370,0.145648,129.96,245.35,195.945,197.25250,39.128154,1531.012421,0.198366,0.0,0.11,101.0,0.67875,107.0
1106,2024-01-04 22:00:00,180.34,215.56,194.300,196.03125,11.146863,124.252555,0.056863,23.29,43.14,26.405,28.51125,6.644468,44.148955,0.233047,89.90,120.01,99.850,99.39000,9.412291,88.591229,0.094701,1.36,1.48,1.370,1.39125,0.041555,0.001727,0.029869,0.154,0.7,24.4,64.300000,46.21,72.17,65.470,64.18250,8.605206,74.049564,0.134074,6.08,143.40,100.

In [ ]:
print(resampled_df.shape)
print(filtered_df.shape)

(1152, 66)
(1107, 66)


In [ ]:
filtered_df = filtered_df.interpolate(method='linear',
                                               limit_direction='forward')
filtered_df = filtered_df.interpolate(method='linear',
                                               limit_direction='backward')

In [ ]:
final_df=filtered_df.copy()

In [ ]:
max_timestamp = final_df['Timestamp_To'].max()
max_timestamp_dt = pd.to_datetime(max_timestamp)
future_timestamp = max_timestamp_dt + timedelta(days=15)
target_date_str = future_timestamp.strftime("%Y-%m-%d %H:%M")

In [ ]:
print(future_timestamp)

2024-01-19 22:00:00


In [ ]:
min_shifts = 45
max_shifts = 83
shifts_per_day = 3

In [ ]:
def calculate_lag_dates(target_date_str, min_shifts, max_shifts, shifts_per_day):
    target_date = datetime.strptime(target_date_str, "%Y-%m-%d %H:%M")
    min_lag_days = min_shifts // shifts_per_day
    max_lag_days = max_shifts // shifts_per_day
    max_lag_extra_shifts = max_shifts % shifts_per_day
    start_date_min_lag = target_date - timedelta(days=min_lag_days)
    start_date_max_lag = target_date - timedelta(days=max_lag_days, hours=max_lag_extra_shifts * 8)
    return start_date_min_lag, start_date_max_lag


In [ ]:
start_date_min_lag, start_date_max_lag = calculate_lag_dates(target_date_str, min_shifts, max_shifts, shifts_per_day)

In [ ]:
print(start_date_min_lag, start_date_max_lag)

2024-01-04 22:00:00 2023-12-23 06:00:00


In [ ]:
print("Start Date for Minimum Lag:", start_date_min_lag.strftime("%Y-%m-%d %H:%M"))
print("Start Date for Maximum Lag:", start_date_max_lag.strftime("%Y-%m-%d %H:%M"))

Start Date for Minimum Lag: 2024-01-04 22:00
Start Date for Maximum Lag: 2023-12-23 06:00


In [ ]:
final_df_lag = final_df[(final_df['Timestamp_To'] <= start_date_min_lag) & (final_df['Timestamp_To'] >= start_date_max_lag)]

In [ ]:
final_df_lag.shape

(39, 66)

In [ ]:
final_df_lag.head()

,Timestamp_To,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50
1068,2023-12-23 06:00:00,167.37,202.16,183.960,184.25875,10.829285,117.273413,0.058772,39.47,45.39,41.315,41.73125,1.828945,3.345041,0.043827,87.67,91.56,90.000,89.90250,1.054741,1.112479,0.011732,1.47,1.49,1.470,1.47500,0.007559,0.000057,0.005125,0.143,0.7,20.1,61.800,54.86,74.43,63.085,64.65625,8.532360,72.801170,0.131965,32.71,201.88,88.585,97.57000,59.560758,3547.483886,0.610441,102.77,158.35,135.470,128.84875,20.030721,401.229784,0.155459,181.82,244.57,219.335,215.24875,21.346883,455.689413,0.099173,0.0,1.8,83.9,0.673000,93.5
1069,2023-12-23 14:00:00,159.11,191.34,175.090,175.81250,12.463561,155.340364,0.070891,40.12,42.73,41.635,41.65875,0.833451,0.694641,0.020007,89.95,90.03,90.005,89.99750,0.026049,0.000679,0.000289,1.47,1.48,1.470,1.47250,0.004629,0.000021,0.003144,0.125,0.5,15.4,64.185,43.88,75.10,65.400,63.37000,11.945194,142.687657,0.188499,0.05,239.50,114.185,115.61375,95.388459,9098.958027,0.825062,87.05,156.05,130.120,127.06000,25.457261,648.072114,0.200356,102.06,231.11,219.350,203.58000,41.919225,1757.221429,0.205910,0.0,0.8,90.7,0.670000,90.1
1070,2023-12-23 22:00:00,183.58,230.74,213.385,208.41375,15.925550,253.623141,0.076413,30.77,41.42,36.675,36.68750,3.694342,13.648164,0.100698,89.84,90.10,90.010,89.99250,0.083281,0.006936,0.000925,1.42,1.47,1.445,1.44750,0.018323,0.000336,0.012658,0.150,0.7,23.0,60.050,56.42,75.03,73.795,71.11750,6.176274,38.146364,0.086846,72.05,247.45,201.780,185.39750,54.001415,2916.152850,0.291274,116.83,159.03,143.075,141.12750,13.883301,192.746050,0.098374,188.68,222.14,210.740,210.23625,10.790392,116.432570,0.051325,0.0,1.1,88.6,0.674667,94.4
1071,2023-12-24 06:00:00,188.26,233.75,232.010,221.59125,18.922222,358.050470,0.085392,22.72,28.27,23.785,24.60375,2.014844,4.059598,0.081892,87.45,90.04,89.995,89.51875,0.955726,0.913412,0.010676,1.37,1.40,1.380,1.38375,0.010607,0.000112,0.007665,0.176,0.8,35.8,52.800,66.78,74.65,72.275,71.76000,2.536792,6.435314,0.035351,72.08,232.48,129.175,136.94000,50.822496,2582.926143,0.371130,100.16,155.69,124.145,126.09000,19.275407,371.541314,0.152870,194.10,220.90,211.245,210.42125,10.046959,100.941384,0.047747,0.0,1.3,87.6,0.666500,94.2
1072,2023-12-24 14:00:00,209.53,233.20,227.015,225.90625,7.655071,58.600113,0.033886,24.87,44.94,32.165,33.95000,6.954838,48.369771,0.204855,86.66,90.08,89.975,89.42750,1.197125,1.433107,0.013387,1.39,1.49,1.425,1.43375,0.035832,0.001284,0.024992,0.178,0.9,36.2,55.365,51.02,72.15,71.650,66.75875,8.192733,67.120870,0.122721,71.55,186.93,111.700,127.39125,46.690109,2179.

In [ ]:
df=final_df_lag.copy()
df.set_index('Timestamp_To', inplace=True)

In [ ]:
for column in df.columns:
    for i in range(45,84):
        nth_last_value = df[column].iloc[-(i - 44)]
        new_column_name = f'{column}_shift_{i}'
        df[new_column_name] = pd.NA
        df.at[df.index[-1],new_column_name] = nth_last_value

In [ ]:
final_df_lag.head()

,Timestamp_To,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50
1068,2023-12-23 06:00:00,167.37,202.16,183.960,184.25875,10.829285,117.273413,0.058772,39.47,45.39,41.315,41.73125,1.828945,3.345041,0.043827,87.67,91.56,90.000,89.90250,1.054741,1.112479,0.011732,1.47,1.49,1.470,1.47500,0.007559,0.000057,0.005125,0.143,0.7,20.1,61.800,54.86,74.43,63.085,64.65625,8.532360,72.801170,0.131965,32.71,201.88,88.585,97.57000,59.560758,3547.483886,0.610441,102.77,158.35,135.470,128.84875,20.030721,401.229784,0.155459,181.82,244.57,219.335,215.24875,21.346883,455.689413,0.099173,0.0,1.8,83.9,0.673000,93.5
1069,2023-12-23 14:00:00,159.11,191.34,175.090,175.81250,12.463561,155.340364,0.070891,40.12,42.73,41.635,41.65875,0.833451,0.694641,0.020007,89.95,90.03,90.005,89.99750,0.026049,0.000679,0.000289,1.47,1.48,1.470,1.47250,0.004629,0.000021,0.003144,0.125,0.5,15.4,64.185,43.88,75.10,65.400,63.37000,11.945194,142.687657,0.188499,0.05,239.50,114.185,115.61375,95.388459,9098.958027,0.825062,87.05,156.05,130.120,127.06000,25.457261,648.072114,0.200356,102.06,231.11,219.350,203.58000,41.919225,1757.221429,0.205910,0.0,0.8,90.7,0.670000,90.1
1070,2023-12-23 22:00:00,183.58,230.74,213.385,208.41375,15.925550,253.623141,0.076413,30.77,41.42,36.675,36.68750,3.694342,13.648164,0.100698,89.84,90.10,90.010,89.99250,0.083281,0.006936,0.000925,1.42,1.47,1.445,1.44750,0.018323,0.000336,0.012658,0.150,0.7,23.0,60.050,56.42,75.03,73.795,71.11750,6.176274,38.146364,0.086846,72.05,247.45,201.780,185.39750,54.001415,2916.152850,0.291274,116.83,159.03,143.075,141.12750,13.883301,192.746050,0.098374,188.68,222.14,210.740,210.23625,10.790392,116.432570,0.051325,0.0,1.1,88.6,0.674667,94.4
1071,2023-12-24 06:00:00,188.26,233.75,232.010,221.59125,18.922222,358.050470,0.085392,22.72,28.27,23.785,24.60375,2.014844,4.059598,0.081892,87.45,90.04,89.995,89.51875,0.955726,0.913412,0.010676,1.37,1.40,1.380,1.38375,0.010607,0.000112,0.007665,0.176,0.8,35.8,52.800,66.78,74.65,72.275,71.76000,2.536792,6.435314,0.035351,72.08,232.48,129.175,136.94000,50.822496,2582.926143,0.371130,100.16,155.69,124.145,126.09000,19.275407,371.541314,0.152870,194.10,220.90,211.245,210.42125,10.046959,100.941384,0.047747,0.0,1.3,87.6,0.666500,94.2
1072,2023-12-24 14:00:00,209.53,233.20,227.015,225.90625,7.655071,58.600113,0.033886,24.87,44.94,32.165,33.95000,6.954838,48.369771,0.204855,86.66,90.08,89.975,89.42750,1.197125,1.433107,0.013387,1.39,1.49,1.425,1.43375,0.035832,0.001284,0.024992,0.178,0.9,36.2,55.365,51.02,72.15,71.650,66.75875,8.192733,67.120870,0.122721,71.55,186.93,111.700,127.39125,46.690109,2179.

In [ ]:
final_df_lag.tail()

,Timestamp_To,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50
1102,2024-01-03 14:00:00,164.30,232.93,183.920,195.84625,28.057475,787.221912,0.143263,23.05,43.94,27.395,29.96125,7.296235,53.235041,0.243522,89.89,90.11,90.050,90.03250,0.078331,0.006136,0.000870,1.37,1.49,1.400,1.41125,0.041209,0.001698,0.029201,0.176,0.9,31.8,67.900000,60.68,73.61,71.880,70.30375,4.429256,19.618313,0.063002,37.12,231.20,95.115,111.00250,62.902935,3956.779250,0.566680,138.25,224.06,195.365,187.57875,30.251320,915.142355,0.161273,157.65,252.64,217.270,213.89125,29.853118,891.208641,0.139571,0.0,0.00,103.0,0.67000,105.0
1103,2024-01-03 22:00:00,150.07,187.79,164.255,167.34875,12.825173,164.485070,0.076637,28.59,36.39,33.845,33.69750,2.498375,6.241879,0.074141,89.00,90.95,90.010,89.99625,0.523776,0.274341,0.005820,1.40,1.45,1.435,1.43250,0.014880,0.000221,0.010388,0.154,0.8,19.9,72.800000,60.94,74.01,70.870,69.03375,5.311010,28.206827,0.076934,60.83,197.63,122.645,127.81625,47.155962,2223.684712,0.368936,157.80,193.42,171.570,173.20500,14.721896,216.734229,0.084997,176.92,254.43,208.335,208.54875,23.309149,543.316413,0.111768,0.0,1.00,93.0,0.66900,112.0
1104,2024-01-04 06:00:00,160.16,206.59,176.500,178.42625,14.855285,220.679484,0.083257,25.05,32.33,30.445,29.49375,2.592489,6.720998,0.087900,88.85,90.08,89.965,89.73875,0.498525,0.248527,0.005555,1.38,1.43,1.410,1.40750,0.016690,0.000279,0.011858,0.168,0.8,28.0,59.000000,63.02,74.88,69.245,69.04500,4.966629,24.667400,0.071933,69.89,133.15,115.035,109.91500,20.714797,429.102829,0.188462,144.05,185.45,157.795,160.10375,13.762329,189.401712,0.085959,107.90,263.98,213.010,202.44875,51.791141,2682.322327,0.255823,0.0,0.00,102.0,0.67450,105.0
1105,2024-01-04 14:00:00,155.95,206.67,172.235,176.65750,17.148238,294.062050,0.097071,22.25,24.86,24.040,23.69375,1.113244,1.239312,0.046985,89.93,110.28,90.060,95.34000,7.874701,62.010914,0.082596,1.36,1.38,1.370,1.37125,0.006409,0.000041,0.004674,0.215,1.1,42.5,54.786667,60.34,77.26,73.855,71.18250,6.664826,44.419907,0.093630,72.26,184.38,107.385,116.31125,42.222794,1782.764298,0.363016,125.88,181.55,147.805,150.48375,21.917718,480.386370,0.145648,129.96,245.35,195.945,197.25250,39.128154,1531.012421,0.198366,0.0,0.11,101.0,0.67875,107.0
1106,2024-01-04 22:00:00,180.34,215.56,194.300,196.03125,11.146863,124.252555,0.056863,23.29,43.14,26.405,28.51125,6.644468,44.148955,0.233047,89.90,120.01,99.850,99.39000,9.412291,88.591229,0.094701,1.36,1.48,1.370,1.39125,0.041555,0.001727,0.029869,0.154,0.7,24.4,64.300000,46.21,72.17,65.470,64.18250,8.605206,74.049564,0.134074,6.08,143.40,100.

In [ ]:
df.head()

,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50,Special_Filling_Flow_min_shift_45,Special_Filling_Flow_min_shift_46,Special_Filling_Flow_min_shift_47,Special_Filling_Flow_min_shift_48,Special_Filling_Flow_min_shift_49,Special_Filling_Flow_min_shift_50,Special_Filling_Flow_min_shift_51,Special_Filling_Flow_min_shift_52,Special_Filling_Flow_min_shift_53,Special_Filling_Flow_min_shift_54,Special_Filling_Flow_min_shift_55,Special_Filling_Flow_min_shift_56,Special_Filling_Flow_min_shift_57,Special_Filling_Flow_min_shift_58,Special_Filling_Flow_min_shift_59,Special_Filling_Flow_min_shift_60,Special_Filling_Flow_min_shift_61,Special_Filling_Flow_min_shift_62,Special_Filling_Flow_min_shift_63,Special_Filling_Flow_min_shift_64,Special_Filling_Flow_min_shift_65,Special_Filling_Flow_min_shift_66,Special_Filling_Flow_min_shift_67,Special_Filling_Flow_min_shift_68,Special_Filling_Flow_min_shift_69,Special_Filling_Flow_min_shift_70,Special_Filling_Flow_min_shift_71,Special_Filling_Flow_min_shift_72,Special_Filling_Flow_min_shift_73,Special_Filling_Flow_min_shift_74,Special_Filling_Flow_min_shift_75,Special_Filling_Flow_min_shift_76,Special_Filling_Flow_min_shift_77,Special_Filling_Flow_min_shift_78,Special_Filling_Flow_min_shift_79,Special_Filling_Flow_min_shift_80,Special_Filling_Flow_min_shift_81,Special_Filling_Flow_min_shift_82,Special_Filling_Flow_min_shift_83,Special_Filling_Flow_max_shift_45,Special_Filling_Flow_max_shift_46,Special_Filling_Flow_max_shift_47,Special_Filling_Flow_max_shift_48,Special_Filling_Flow_max_shift_49,Special_Filling_Flow_max_shift_50,Special_Filling_Flow_max_shift_51,Special_Filling_Flow_max_shift_52,Special_Filling_Flow_max_shift_53,Special_Filling_Flow_max_shift_54,Special_Filling_Flow_max_shift_55,Special_Filling_Flow_max_shift_56,Special_Filling_Flow_max_shift_57,Special_Filling_Flow_max_shift_58,Special_Filling_Flow_max_shift_59,Special_Filling_Flow_max_shift_60,Special_Filling_Flow_max_shift_61,Special_Filling_Flow_max_shift_62,Special_Filling_Flow_max_shift_63,Special_Filling_Flow_max_shift_64,Special_Filling_Flow_max_shift_65,Special_Filling_Flow_max_shift_66,Special_Filling_Flow_max_shift_67,Special_Filling_Flow_max_shift_68,Special_Filling_Flow_max_shift_69,Special_Filling_Flow_max_shift_70,Special_Filling_Flow_max_shift_71,Special_Filling_Flow_max_shift_72,Special_Filling_Flow_max_shift_73,Special_Filling_Flow_max_shift_74,Special_Filling_Flow_max_shift_75,Special_Filling_Flow_max_shift_76,Special_Filling_Flow_max_shift_77,Special_Filling_Flow_max_shift_78,Special_Filling_Flow_max_shift_79,Special_Filling_Flow_ma

In [ ]:
df.tail()

,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50,Special_Filling_Flow_min_shift_45,Special_Filling_Flow_min_shift_46,Special_Filling_Flow_min_shift_47,Special_Filling_Flow_min_shift_48,Special_Filling_Flow_min_shift_49,Special_Filling_Flow_min_shift_50,Special_Filling_Flow_min_shift_51,Special_Filling_Flow_min_shift_52,Special_Filling_Flow_min_shift_53,Special_Filling_Flow_min_shift_54,Special_Filling_Flow_min_shift_55,Special_Filling_Flow_min_shift_56,Special_Filling_Flow_min_shift_57,Special_Filling_Flow_min_shift_58,Special_Filling_Flow_min_shift_59,Special_Filling_Flow_min_shift_60,Special_Filling_Flow_min_shift_61,Special_Filling_Flow_min_shift_62,Special_Filling_Flow_min_shift_63,Special_Filling_Flow_min_shift_64,Special_Filling_Flow_min_shift_65,Special_Filling_Flow_min_shift_66,Special_Filling_Flow_min_shift_67,Special_Filling_Flow_min_shift_68,Special_Filling_Flow_min_shift_69,Special_Filling_Flow_min_shift_70,Special_Filling_Flow_min_shift_71,Special_Filling_Flow_min_shift_72,Special_Filling_Flow_min_shift_73,Special_Filling_Flow_min_shift_74,Special_Filling_Flow_min_shift_75,Special_Filling_Flow_min_shift_76,Special_Filling_Flow_min_shift_77,Special_Filling_Flow_min_shift_78,Special_Filling_Flow_min_shift_79,Special_Filling_Flow_min_shift_80,Special_Filling_Flow_min_shift_81,Special_Filling_Flow_min_shift_82,Special_Filling_Flow_min_shift_83,Special_Filling_Flow_max_shift_45,Special_Filling_Flow_max_shift_46,Special_Filling_Flow_max_shift_47,Special_Filling_Flow_max_shift_48,Special_Filling_Flow_max_shift_49,Special_Filling_Flow_max_shift_50,Special_Filling_Flow_max_shift_51,Special_Filling_Flow_max_shift_52,Special_Filling_Flow_max_shift_53,Special_Filling_Flow_max_shift_54,Special_Filling_Flow_max_shift_55,Special_Filling_Flow_max_shift_56,Special_Filling_Flow_max_shift_57,Special_Filling_Flow_max_shift_58,Special_Filling_Flow_max_shift_59,Special_Filling_Flow_max_shift_60,Special_Filling_Flow_max_shift_61,Special_Filling_Flow_max_shift_62,Special_Filling_Flow_max_shift_63,Special_Filling_Flow_max_shift_64,Special_Filling_Flow_max_shift_65,Special_Filling_Flow_max_shift_66,Special_Filling_Flow_max_shift_67,Special_Filling_Flow_max_shift_68,Special_Filling_Flow_max_shift_69,Special_Filling_Flow_max_shift_70,Special_Filling_Flow_max_shift_71,Special_Filling_Flow_max_shift_72,Special_Filling_Flow_max_shift_73,Special_Filling_Flow_max_shift_74,Special_Filling_Flow_max_shift_75,Special_Filling_Flow_max_shift_76,Special_Filling_Flow_max_shift_77,Special_Filling_Flow_max_shift_78,Special_Filling_Flow_max_shift_79,Special_Filling_Flow_ma

In [ ]:
df_lag_cleaned = df.dropna()

In [ ]:
df_lag_cleaned.shape

(1, 2600)

In [ ]:
df_lag_cleaned.head()

,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50,Special_Filling_Flow_min_shift_45,Special_Filling_Flow_min_shift_46,Special_Filling_Flow_min_shift_47,Special_Filling_Flow_min_shift_48,Special_Filling_Flow_min_shift_49,Special_Filling_Flow_min_shift_50,Special_Filling_Flow_min_shift_51,Special_Filling_Flow_min_shift_52,Special_Filling_Flow_min_shift_53,Special_Filling_Flow_min_shift_54,Special_Filling_Flow_min_shift_55,Special_Filling_Flow_min_shift_56,Special_Filling_Flow_min_shift_57,Special_Filling_Flow_min_shift_58,Special_Filling_Flow_min_shift_59,Special_Filling_Flow_min_shift_60,Special_Filling_Flow_min_shift_61,Special_Filling_Flow_min_shift_62,Special_Filling_Flow_min_shift_63,Special_Filling_Flow_min_shift_64,Special_Filling_Flow_min_shift_65,Special_Filling_Flow_min_shift_66,Special_Filling_Flow_min_shift_67,Special_Filling_Flow_min_shift_68,Special_Filling_Flow_min_shift_69,Special_Filling_Flow_min_shift_70,Special_Filling_Flow_min_shift_71,Special_Filling_Flow_min_shift_72,Special_Filling_Flow_min_shift_73,Special_Filling_Flow_min_shift_74,Special_Filling_Flow_min_shift_75,Special_Filling_Flow_min_shift_76,Special_Filling_Flow_min_shift_77,Special_Filling_Flow_min_shift_78,Special_Filling_Flow_min_shift_79,Special_Filling_Flow_min_shift_80,Special_Filling_Flow_min_shift_81,Special_Filling_Flow_min_shift_82,Special_Filling_Flow_min_shift_83,Special_Filling_Flow_max_shift_45,Special_Filling_Flow_max_shift_46,Special_Filling_Flow_max_shift_47,Special_Filling_Flow_max_shift_48,Special_Filling_Flow_max_shift_49,Special_Filling_Flow_max_shift_50,Special_Filling_Flow_max_shift_51,Special_Filling_Flow_max_shift_52,Special_Filling_Flow_max_shift_53,Special_Filling_Flow_max_shift_54,Special_Filling_Flow_max_shift_55,Special_Filling_Flow_max_shift_56,Special_Filling_Flow_max_shift_57,Special_Filling_Flow_max_shift_58,Special_Filling_Flow_max_shift_59,Special_Filling_Flow_max_shift_60,Special_Filling_Flow_max_shift_61,Special_Filling_Flow_max_shift_62,Special_Filling_Flow_max_shift_63,Special_Filling_Flow_max_shift_64,Special_Filling_Flow_max_shift_65,Special_Filling_Flow_max_shift_66,Special_Filling_Flow_max_shift_67,Special_Filling_Flow_max_shift_68,Special_Filling_Flow_max_shift_69,Special_Filling_Flow_max_shift_70,Special_Filling_Flow_max_shift_71,Special_Filling_Flow_max_shift_72,Special_Filling_Flow_max_shift_73,Special_Filling_Flow_max_shift_74,Special_Filling_Flow_max_shift_75,Special_Filling_Flow_max_shift_76,Special_Filling_Flow_max_shift_77,Special_Filling_Flow_max_shift_78,Special_Filling_Flow_max_shift_79,Special_Filling_Flow_ma

In [ ]:
best_cols = '/content/precip_final_cols (1).pkl'
with open(best_cols, 'rb') as file:
    final_cols = pickle.load(file)

In [ ]:
final_cols

['Fine_Seed_Tonnage_min_shift_43',
 'Fine_Seed_Tonnage_min_shift_50',
 'Fine_Seed_Tonnage_min_shift_59',
 'Fine_Seed_Tonnage_min_shift_61',
 'Fine_Seed_Tonnage_max_shift_42',
 'Fine_Seed_Tonnage_max_shift_62',
 'Fine_Seed_Tonnage_median_shift_42',
 'Fine_Seed_Flow_max_shift_62',
 'Fine_Seed_Density_mean_shift_74',
 'Fine_Seed_Density_mean_shift_76',
 'Fine_Seed_Density_mean_shift_82',
 'Fine_Seed_Density_mean_shift_83',
 'Fine_Seed_Density_var_coeff_shift_60',
 'Fine_Seed_SSA_shift_57',
 'Fine_Seed_SSA_shift_70',
 'Fine_Seed_45u_shift_39',
 'Fine_Seed_45u_shift_45',
 'Fine_Seed_45u_shift_55',
 'Fine_Seed_45u_shift_57',
 'Fine_Seed_45u_shift_77',
 'Fine_Seed_45u_shift_81',
 'Fine_Seed_45u_shift_83',
 'Fine_Seed_D50_shift_61',
 'Fine_Seed_D50_shift_66',
 'Fine_Seed_D50_shift_82',
 'PHE_Inlet_Temp_min_shift_41',
 'PHE_Inlet_Temp_min_shift_73',
 'PHE_Inlet_Temp_std_shift_42',
 'PHE_Inlet_Temp_std_shift_52',
 'PHE_Inlet_Temp_std_shift_54',
 'PHE_Inlet_Temp_std_shift_59',
 'PHE_Inlet_Temp_st

In [ ]:
filtered_cols = [col for col in final_cols if col != "Feed_Hydrate_D50" and int(col.split('_')[-1]) >= 45]

filtered_cols.append("Feed_Hydrate_D50")

In [ ]:
filtered_cols

['Fine_Seed_Tonnage_min_shift_50',
 'Fine_Seed_Tonnage_min_shift_59',
 'Fine_Seed_Tonnage_min_shift_61',
 'Fine_Seed_Tonnage_max_shift_62',
 'Fine_Seed_Flow_max_shift_62',
 'Fine_Seed_Density_mean_shift_74',
 'Fine_Seed_Density_mean_shift_76',
 'Fine_Seed_Density_mean_shift_82',
 'Fine_Seed_Density_mean_shift_83',
 'Fine_Seed_Density_var_coeff_shift_60',
 'Fine_Seed_SSA_shift_57',
 'Fine_Seed_SSA_shift_70',
 'Fine_Seed_45u_shift_45',
 'Fine_Seed_45u_shift_55',
 'Fine_Seed_45u_shift_57',
 'Fine_Seed_45u_shift_77',
 'Fine_Seed_45u_shift_81',
 'Fine_Seed_45u_shift_83',
 'Fine_Seed_D50_shift_61',
 'Fine_Seed_D50_shift_66',
 'Fine_Seed_D50_shift_82',
 'PHE_Inlet_Temp_min_shift_73',
 'PHE_Inlet_Temp_std_shift_52',
 'PHE_Inlet_Temp_std_shift_54',
 'PHE_Inlet_Temp_std_shift_59',
 'PHE_Inlet_Temp_std_shift_62',
 'PHE_Inlet_Temp_std_shift_78',
 'Regular_Filling_Flow_min_shift_45',
 'Regular_Filling_Flow_min_shift_50',
 'Regular_Filling_Flow_max_shift_45',
 'Regular_Filling_Flow_max_shift_48',
 '

In [ ]:
df_final_cleaned=df_lag_cleaned[filtered_cols]

In [ ]:
df_final_cleaned.shape

(1, 70)

In [ ]:
target_column ='Feed_Hydrate_D50'
X_unseen = df_final_cleaned.drop([target_column],axis=1).copy()
y_unseen = df_final_cleaned[[target_column]].copy()
best_model = '/content/Light_gbm_model_04-01-2024.pkl'

with open(best_model, 'rb') as file:
    model = pickle.load(file)


In [ ]:
X_unseen

,Fine_Seed_Tonnage_min_shift_50,Fine_Seed_Tonnage_min_shift_59,Fine_Seed_Tonnage_min_shift_61,Fine_Seed_Tonnage_max_shift_62,Fine_Seed_Flow_max_shift_62,Fine_Seed_Density_mean_shift_74,Fine_Seed_Density_mean_shift_76,Fine_Seed_Density_mean_shift_82,Fine_Seed_Density_mean_shift_83,Fine_Seed_Density_var_coeff_shift_60,Fine_Seed_SSA_shift_57,Fine_Seed_SSA_shift_70,Fine_Seed_45u_shift_45,Fine_Seed_45u_shift_55,Fine_Seed_45u_shift_57,Fine_Seed_45u_shift_77,Fine_Seed_45u_shift_81,Fine_Seed_45u_shift_83,Fine_Seed_D50_shift_61,Fine_Seed_D50_shift_66,Fine_Seed_D50_shift_82,PHE_Inlet_Temp_min_shift_73,PHE_Inlet_Temp_std_shift_52,PHE_Inlet_Temp_std_shift_54,PHE_Inlet_Temp_std_shift_59,PHE_Inlet_Temp_std_shift_62,PHE_Inlet_Temp_std_shift_78,Regular_Filling_Flow_min_shift_45,Regular_Filling_Flow_min_shift_50,Regular_Filling_Flow_max_shift_45,Regular_Filling_Flow_max_shift_48,Regular_Filling_Flow_max_shift_51,Regular_Filling_Flow_max_shift_57,Regular_Filling_Flow_max_shift_70,Regular_Filling_Flow_max_shift_77,Regular_Filling_Flow_max_shift_82,Regular_Filling_Flow_median_shift_52,Regular_Filling_Flow_median_shift_57,Regular_Filling_Flow_std_shift_49,Regular_Filling_Flow_std_shift_73,Regular_Filling_Flow_std_shift_74,Regular_Filling_Flow_std_shift_75,Regular_Filling_Flow_std_shift_77,Regular_Filling_Flow_std_shift_78,Regular_Filling_Flow_std_shift_82,Regular_Filling_Flow_var_coeff_shift_58,Regular_Filling_Flow_var_coeff_shift_70,Coarse_Seed_2_Flow_max_shift_52,Coarse_Seed_2_Flow_mean_shift_54,HAT_45u_shift_47,HAT_45u_shift_49,HAT_45u_shift_50,HAT_45u_shift_56,HAT_45u_shift_67,HAT_45u_shift_78,HAT_D50_shift_45,HAT_D50_shift_48,HAT_D50_shift_51,HAT_D50_shift_54,HAT_D50_shift_60,HAT_D50_shift_61,HAT_D50_shift_68,HAT_D50_shift_69,HAT_D50_shift_71,HAT_D50_shift_79,HAT_D50_shift_81,Filling_Ratio_shift_54,Filling_Ratio_shift_64,Filling_Ratio_shift_80
Timestamp_To,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-01-04 22:00:00,48.62,33.95,49.1,46.81,96.06,1.48,1.5025,1.4725,1.475,0.024467,0.1315,0.135,24.4,18.6,17.05,5.5,23.0,20.1,81.4,59.35,64.185,66.59,0.77807,1.858094,0.748594,5.993756,10.297743,6.08,58.74,143.4,197.63,205.98,148.05,178.44,191.84,239.5,76.805,105.47,62.902935,79.046261,61.541633,54.370907,78.43993,98.352788,95.388459,0.227219,0.662841,219.88,245.99375,0.0,0.0,0.35,0.08,0.2,1.7,101.0,93.0,95.0,103.0,97.1,90.8,96.6,96.6,85.7,96.1,88.6,0.68,0.63975,0.6665


In [ ]:
list(X_unseen.columns)

['Fine_Seed_Tonnage_min_shift_50',
 'Fine_Seed_Tonnage_min_shift_59',
 'Fine_Seed_Tonnage_min_shift_61',
 'Fine_Seed_Tonnage_max_shift_62',
 'Fine_Seed_Flow_max_shift_62',
 'Fine_Seed_Density_mean_shift_74',
 'Fine_Seed_Density_mean_shift_76',
 'Fine_Seed_Density_mean_shift_82',
 'Fine_Seed_Density_mean_shift_83',
 'Fine_Seed_Density_var_coeff_shift_60',
 'Fine_Seed_SSA_shift_57',
 'Fine_Seed_SSA_shift_70',
 'Fine_Seed_45u_shift_45',
 'Fine_Seed_45u_shift_55',
 'Fine_Seed_45u_shift_57',
 'Fine_Seed_45u_shift_77',
 'Fine_Seed_45u_shift_81',
 'Fine_Seed_45u_shift_83',
 'Fine_Seed_D50_shift_61',
 'Fine_Seed_D50_shift_66',
 'Fine_Seed_D50_shift_82',
 'PHE_Inlet_Temp_min_shift_73',
 'PHE_Inlet_Temp_std_shift_52',
 'PHE_Inlet_Temp_std_shift_54',
 'PHE_Inlet_Temp_std_shift_59',
 'PHE_Inlet_Temp_std_shift_62',
 'PHE_Inlet_Temp_std_shift_78',
 'Regular_Filling_Flow_min_shift_45',
 'Regular_Filling_Flow_min_shift_50',
 'Regular_Filling_Flow_max_shift_45',
 'Regular_Filling_Flow_max_shift_48',
 '

In [ ]:
X_unseen.index + timedelta(days=15)

DatetimeIndex(['2024-01-19 22:00:00'], dtype='datetime64[ns]', name='Timestamp_To', freq=None)

In [ ]:
preds = model.predict(X_unseen.values)

In [ ]:
print(f"we are predicting for {future_timestamp}")

we are predicting for 2024-01-19 22:00:00


In [ ]:
future_timestamp

Timestamp('2024-01-19 22:00:00')

In [ ]:
preds

array([108.7532109])

In [ ]:
preds

array([108.7532109])

In [ ]:
preds=preds.round(2)

In [ ]:
df_unseen_PSD=pd.DataFrame()
df_unseen_PSD['Timestamp_To']=X_unseen.index+ timedelta(days=15)
df_unseen_PSD['PSD_Prediction']=preds.round(2)

In [ ]:
df_unseen_PSD

,Timestamp_To,PSD_Prediction
0,2024-01-19 22:00:00,108.75


In [ ]:
df_unseen_PSD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Timestamp_To    1 non-null      datetime64[ns]
 1   PSD_Prediction  1 non-null      float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 148.0 bytes


In [ ]:
def get_predict_shift(df):
    hour_now=df['Timestamp_To'].dt.hour[0]
    if hour_now < 14:  # Before 2 PM
        if hour_now < 6:  # Before 6 AM
            shift = "C"  # Previous day's night shift
        else:
            shift = "A"  # Morning shift
    elif hour_now < 22:  # Between 2 PM and 10 PM
        shift = "B"  # Afternoon shift
    else:
        shift = "C"  # Night shift
    predicted_hour=df['Timestamp_To'][0]
    return shift,predicted_hour


In [ ]:
predict_shift,predicted_hour=get_predict_shift(df_unseen_PSD)

In [ ]:
predict_shift

'C'

In [ ]:
predicted_hour

Timestamp('2024-01-19 22:00:00')

In [ ]:
ped=df_unseen_PSD['PSD_Prediction'][0]

In [ ]:
ped

108.75